In [11]:
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import folium
import numpy as np

def find_optimal_living_area(residential_type, num_living_areas=3):
    # Load data
    data = pd.read_csv('relevant_buildings.csv')

    # Create 'Importance' holding the weights
    data['Importance'] = data['New_Category'].map(weights)

    # Filter the data by 'residential_type'
    buildings_df = data[data["New_Category"] == residential_type]

    # If buildings_df is empty, print a message and return
    if buildings_df.empty:
        print(f"No buildings of type '{residential_type}' found in the data.")
        return

    # Create the map
    map = folium.Map(location=[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=12)

    # Apply KMeans for each category and compute the weighted center
    centers_df = data.groupby('New_Category').apply(lambda df: pd.DataFrame(KMeans(n_clusters=min(num_living_areas, len(df)), n_init=10, random_state=0)

    .fit(df[['Latitude', 'Longitude']])
    .cluster_centers_, columns=['Latitude', 'Longitude'])).reset_index().drop(columns='level_1')

    icon_dict = {
        'food_beverage': 'cutlery',
        'market': 'info-sign',
        'service': 'wrench',
        'leisure': 'leaf',
        'entertainment': 'music',
        'education': 'book',
        'public': 'building-o'
    }

    # Create a NearestNeighbors model
    nbrs = NearestNeighbors(n_neighbors=5).fit(buildings_df[["Latitude", "Longitude"]])

    # Calculate the distance and indices of the nearest neighbors to each center of the selected residential type
    residential_centers_df = centers_df[centers_df['New_Category'] == residential_type]
    distances, indices = nbrs.kneighbors(residential_centers_df[["Latitude", "Longitude"]])

    residential_centers_df = residential_centers_df.assign(indices=list(indices))

    # Add markers to the map for each center
    centers_df_to_add = centers_df[~centers_df['New_Category'].isin(['house', 'residential', 'apartments'])]
    if residential_type in ['house', 'residential', 'apartments']:
        centers_df_to_add = pd.concat([centers_df_to_add, residential_centers_df])
        
    centers_df_to_add.apply(lambda row: folium.Marker(location=[row.Latitude, row.Longitude], 
                                        popup=f"{row.New_Category} center", 
                                        icon=folium.Icon(icon=icon_dict.get(row.New_Category, 'cloud'), prefix='fa')
                                        ).add_to(map), axis=1)
    
    # Add markers for the optimal living area corresponding to the residential type
    optimal_coords_df = buildings_df.iloc[np.hstack(indices)][['Latitude', 'Longitude']]
    optimal_coords_df.apply(lambda coord: folium.Marker(location=[coord.Latitude, coord.Longitude], 
                                        popup="Optimal living area", 
                                        icon=folium.Icon(icon="home", color='red', prefix='fa')
                                        ).add_to(map), axis=1)

    # Display the map
    return map




In [12]:
# Define weights for each category
weights = {'food_beverage': 2, 'market': 1, 'service': 1, 
                'leisure': 2, 'entertainment': 1, 
               'education': 3, 'public': 1}

find_optimal_living_area('apartments')  # replace 'house' with 'apartments' or 'residential' as needed

c:\Users\13677\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
c:\Users\13677\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\13677\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\13677\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Window